In [17]:
import numpy as np
import pandas as pd # standard as pd, 
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
import warnings 
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
warnings.filterwarnings('ignore')

from pathlib import Path
import os

In [11]:
DATASET = 'Churn_Modelling.csv'
TARGET = 'Exited'
TEST_FILE = 'test_data.csv'

MODEL_NAME = 'classification.pkl'
# SAVE_MODEL_PATH = os.path.join(PACKAGE_ROOT,'trained_models')

### Final fearues used in the model
FEATURES = ['CreditScore',
            'Geography', 
            'Gender',
            'Age',
            'Tenure',
            'Balance', 
            'NumOfProducts',
            'HasCrCard', 
            'IsActiveMember',
            'EstimatedSalary']
PRE_FEATURES = ['RowNumber','CustomerId','Surname']
NUM_FEATUES = ['CreditScore','Age',
               'Tenure','Balance',
               'NumOfProducts','EstimatedSalary']
CAT_FEATUES = ['Geography','Gender',
               'HasCrCard','IsActiveMember']

FEATURES_TO_ENCODE = ['Geography','Gender']
FEATURES_TO_SCALE = ['CreditScore','Age',
                     'Tenure','Balance',
                     'NumOfProducts','EstimatedSalary']

FEATURES_DROP = ['RowNumber','CustomerId','Surname']

In [10]:
# dataset = pd.read_csv('Churn_Modelling.csv')
def load_dataset(filepath = 'Churn_Modelling.csv'):
    # filepath = ''
    _data = pd.read_csv(filepath)
    _data.columns = [c.strip() for c in _data.columns] 
    return _data
data = load_dataset()


In [13]:
def separate_data(data):
    X = data.drop(TARGET, axis=1)
    y = data[TARGET]
    return X,y
X, y = separate_data(data)

In [18]:
def split_data(X, y, test_size=0.2, random_state=42):
  # Split into training and testing sets
  X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=random_state)
  return X_train, X_test, y_train, y_test
X_train, X_test, y_train, y_test = split_data(X, y, test_size=0.2, random_state=42)

In [21]:
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression


In [23]:
from sklearn.base import BaseEstimator, TransformerMixin
class DropColumns(BaseEstimator, TransformerMixin):
    def __init__(self, variables_to_drop=None):
        self.variables_to_drop = variables_to_drop or config.FEATURES_DROP
    
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        X = X.drop(columns=self.variables_to_drop)
        return X

# Transformer to encode and create dummy variables
class EncodeAndBind(BaseEstimator, TransformerMixin):
    def __init__(self, encode=None, dummy=None):
        self.encode = encode
        self.dummy = dummy
    
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        X[self.encode].replace({'Male': 0, 'Female': 1}, inplace=True)
        X = pd.get_dummies(X, columns=[self.dummy])
        X.replace({True: 1, False: 0}, inplace=True)
        return X

# Transformer to normalize specified variables
class Scale(BaseEstimator, TransformerMixin):
    def __init__(self, variables=None):
        self.variables = variables or config.FEATURES_TO_SCALE
    
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        X = X.copy()
        for variable in self.variables:
            X[variable] = (X[variable] - X[variable].min()) / (X[variable].max() - X[variable].min())
        return X


In [24]:
classification_pipeline = Pipeline(
    steps =  [(
    'Drop_columns', DropColumns(variables_to_drop =['RowNumber','CustomerId','Surname'])),
    ('Encode_and_bind', EncodeAndBind(encode='Gender', dummy='Geography')),
    ('Scale', Scale(variables=['CreditScore','Age','Tenure','Balance','NumOfProducts','EstimatedSalary'])),
    ('Model', LogisticRegression(random_state=12))])

In [26]:
classification_pipeline.fit(X_train,y_train)

Pipeline(steps=[('Drop_columns',
                 DropColumns(variables_to_drop=['RowNumber', 'CustomerId',
                                                'Surname'])),
                ('Encode_and_bind',
                 EncodeAndBind(dummy='Geography', encode='Gender')),
                ('Scale',
                 Scale(variables=['CreditScore', 'Age', 'Tenure', 'Balance',
                                  'NumOfProducts', 'EstimatedSalary'])),
                ('Model', LogisticRegression(random_state=12))])

In [32]:
import joblib
def perform_pipeline():
    dataset = load_dataset('Churn_Modelling.csv')
    X,y = separate_data(dataset)
    X_train, X_test, y_train, y_test = split_data(X,y)
    test_data = X_test.copy()
    test_data[TARGET] = y_test
    test_data.to_csv('Testing1.csv', index = False)
    try:
        classification_pipeline.fit(X_train, y_train)
    except Exception as e:
        print(f"Error during pipeline fitting: {e}")

    joblib.dump(classification_pipeline,'classification.pkl')
    print(f"Model has been saved to ")
if __name__=='__main__':
    perform_pipeline()


An unexpected error occurred: 'NoneType' object is not iterable
